## 1. Install & Import Libraries

In [1]:
# Install required libraries and import necessary modules
!pip install transformers datasets

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset


## 2. Load Dataset

In [3]:
# Load the tokenizer and Mahatma Gandhi text dataset
from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load your text file
with open('MahatmaGandhi.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Tokenize the text
tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)



## 3. Tokenization

In [4]:
# Set up model, custom dataset class, and data collator
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from torch.utils.data import Dataset

# Define a custom dataset
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings['input_ids']
        self.attn_masks = encodings['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attn_masks[idx],
            'labels': self.input_ids[idx]
        }

# Load the model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# Prepare dataset
dataset = TextDataset(tokens)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## 4. Model & Dataset Setup

In [5]:
# Define training arguments and start model fine-tuning
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",          # Where to save model
    overwrite_output_dir=True,       # Overwrite previous results
    num_train_epochs=3,              # You can increase if you want
    per_device_train_batch_size=1,   # Small batch for quick training
    save_steps=10_000,
    save_total_limit=1,
    logging_steps=100,
    logging_dir="./logs",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Start training
trainer.train()



`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=3.1674416859944663, metrics={'train_runtime': 60.2806, 'train_samples_per_second': 0.05, 'train_steps_per_second': 0.05, 'total_flos': 783876096000.0, 'train_loss': 3.1674416859944663, 'epoch': 3.0})

## 5. Training

In [6]:
# Generate text using the trained model based on a prompt
prompt = "Mahatma Gandhi believed in"

output = model.generate(
    **tokenizer(prompt, return_tensors='pt'),
    max_length=60,
    num_return_sequences=1,
    do_sample=True,               # Enable randomness
    top_k=50,                     # Sample from top 50 tokens
    top_p=0.95,                   # Nucleus sampling
    temperature=0.9               # Add more creativity
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Mahatma Gandhi believed in a single-mindedness, and the Mahatma Gandhi had given the Buddha a place in his life, and was thus the source of the Mahatma Gandhi's life.

Mahatma Gandhi was born in the village of Kurukshet in Uttar


## 6. Text Generation

In [7]:
# Evaluate model performance using perplexity
model.save_pretrained("./gandhi_nextword_model")
tokenizer.save_pretrained("./gandhi_nextword_model")


('./gandhi_nextword_model/tokenizer_config.json',
 './gandhi_nextword_model/special_tokens_map.json',
 './gandhi_nextword_model/vocab.json',
 './gandhi_nextword_model/merges.txt',
 './gandhi_nextword_model/added_tokens.json')

## 7. Evaluation - Perplexity

In [8]:
import math
import torch

def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    input_ids = inputs["input_ids"]
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
    return math.exp(loss.item())

# Example
model.eval()
perplexity = calculate_perplexity(model, tokenizer, "Mahatma Gandhi was known for")
print(f"Perplexity: {perplexity:.2f}")



Perplexity: 41.59
